# Scenario:
# When encoding a mask with the provided functions (see other notebooks) and decoding it back to an mask, the mask doesnt look like its input?
# only if I change it to delete the transpose command, then it looks as it should.
# Would be nice to hear your thoughts?
# Is it okay, because the competitions hosts are encoding the masks in a other as we do our trainig masks?
# Thx

In [ ]:
import os
import skimage
import numpy as np
import pandas as pd
import skimage.segmentation
import matplotlib.pyplot as plt
from skimage.morphology import label as labeling
import random

# Functions

- functions of other notebooks

In [ ]:
def decode(rle_mask, shape):

    """
    Decode run-length encoded segmentation mask string into 2d array
    Parameters
    ----------
    rle_mask (str): Run-length encoded segmentation mask string
    shape (tuple): Height and width of the mask
    Returns
    -------
    mask [numpy.ndarray of shape (height, width)]: Decoded 2d segmentation mask
    """
    rle_mask = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (rle_mask[0:][::2], rle_mask[1:][::2])]
    starts -= 1
    ends = starts + lengths

    mask = np.zeros((shape[0] * shape[1]), dtype=np.uint8)
    
    for start, end in zip(starts, ends):
        mask[start:end] = 1

    mask = mask.reshape(shape[0], shape[1], 1)
    return mask


def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0] # this is as we can find it in several succesful notebooks
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def rle_encoding_no_t(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = labeling(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
    
def prob_to_rles_no_t(x, cutoff=0.5):
    lab_img = labeling(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding_no_t(lab_img == i)
        

## Ground truth Example

In [ ]:
example = np.array([[0,1,1,1,1,1,1] , [0,0,0,0,0,0,0] , [0,1,1,1,1,1,1] , [0,1,1,1,1,1,1] , [0,1,1,1,1,1,1] ])
print(example)
print(example.shape)
plt.figure(figsize=(15, 10))
plt.imshow(example, cmap = "gray")
plt.axis(False)
plt.show()

# Example Run Length De- / Encoding with Transpose

In [ ]:
def get_submission():

    
    # submission df
    sample_submission = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
    df_submission = pd.DataFrame(data = None, columns = sample_submission.columns)

    # lits for submission df
    list_test_ids = []
    list_rles = []

    count = 0
    id_ = "example"


    # store rles and ids in lists for df 
    rle = list(prob_to_rles(example))      
    list_test_ids.extend([id_] * len(rle)) # extend id for every class in its masks
    list_rles.extend(rle)

    # store lists in df submission
    df_submission['id'] = list_test_ids
    df_submission['predicted'] = pd.Series(list_rles).apply(lambda x: ' '.join(str(y) for y in x))

    # write submission to csv
    df_submission.to_csv('submission.csv', index = False)
    df_test = pd.read_csv('submission.csv')
    
    print(df_test)
    
get_submission()

In [ ]:
df_test = pd.read_csv('submission.csv')

print(example.shape)
mask1  = example 
print(mask1)
print(mask1.shape)
mask__1 = np.zeros((5, 7, 1))
print("")

id_ = "example"
masks_df_id = df_test[df_test["id"]==id_]
print(masks_df_id)          

#loop over annotations per id
for _, row in masks_df_id.iterrows():    
    mask__1 += decode(row["predicted"], (5,7,1))
    
print("")
print("outputted mask:", mask__1[:,:,0])

print("")
plt.figure(figsize=(15, 10))
plt.imshow(mask__1, cmap = "gray")
plt.axis(False)
plt.show()


- outputted mask from run length is destroyed

# Example Run Length De- / Encoding no Transpose

In [ ]:
def get_submission():

    
    # submission df
    sample_submission = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')
    df_submission = pd.DataFrame(data = None, columns = sample_submission.columns)

    # lits for submission df
    list_test_ids = []
    list_rles = []

    count = 0
    id_ = "example"


    # store rles and ids in lists for df 
    rle = list(prob_to_rles_no_t(example))      
    list_test_ids.extend([id_] * len(rle)) # extend id for every class in its masks
    list_rles.extend(rle)

    # store lists in df submission
    df_submission['id'] = list_test_ids
    df_submission['predicted'] = pd.Series(list_rles).apply(lambda x: ' '.join(str(y) for y in x))

    # write submission to csv
    df_submission.to_csv('submission.csv', index = False)
    df_test = pd.read_csv('submission.csv')
    
    print(df_test)
    
get_submission()

In [ ]:
df_test = pd.read_csv('submission.csv')

print(example.shape)
mask2  = example 
print(mask2)
print(mask2.shape)
print("")
mask__2 = np.zeros((5, 7, 1))

masks_df_id = df_test[df_test["id"]==id_]
print(masks_df_id)          

#loop over annotations per id
for _, row in masks_df_id.iterrows():    
    mask__2 += decode(row["predicted"], (5,7,1))

print("outputted mask:", mask__2[:,:,0])

print("")
plt.figure(figsize=(15, 10))
plt.imshow(mask__2, cmap = "gray")
plt.axis(False)
plt.show()

- outputted mask from run length as expected